# Calculating Forest Loss (using Hansen)

In [1]:
import os
import ee
import geemap 

#ee.Authenticate()
ee.Initialize()

## Loading & Clipping Imagery

In [ ]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_hansen = ee.ImageCollection("UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1").mosaic().selfMask()
primaryForest_hansen_COL = primaryForest_hansen.clip(colombiaMpios)

# Clip the Forest loss image to Colombia's boundaries
flossHansen_COL = flossHansen.clip(colombiaMpios)

# Visualization parameters for the map
# viz_params = {
#     'min':1,  # Minimum value for visualization
#     'max': 1,  # Maximum value for visualization
#     'palette': ['008000']  # Color gradient
# }

# Initialize the map using geemap
# Map = geemap.Map(center=[4.0, -72.0], zoom=5)  # Center roughly in Colombia with an appropriate zoom level
# Map.addLayer(primaryForest_hansen_COL, viz_params, "Primary Tropical Forests")
# Map


In [ ]:
# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_hansen_COL.eq(1)

# Masking the floss imagery using the primary mask (cover equals 1)
flossHansen_COLmasked = flossHansen_COL.updateMask(primaryMask)

# Select the 'lossyear' band from the masked forest loss image
loss_year = flossHansen_COLmasked.select(['lossyear'])

# # Visualization parameters
# primary_forest_viz = {
#     'min': 1,
#     'max': 1,
#     'palette': ['green']  # Primary forest is displayed in green
# }

# loss_year_viz = {
#     'min': 1,
#      'max': 1,
#   'palette': ['red']
# }

# # Initialize the map using geemap
# Map = geemap.Map(center=[4.0, -72.0], zoom=6)  # Center roughly in Colombia

# # Add the Primary Tropical Forests layer
# Map.addLayer(primaryForest_hansen_COL, primary_forest_viz, "Primary Tropical Forests")

# # Add the Forest Loss for the specified year layer
# Map.addLayer(modified_loss_year, loss_year_viz, f"Forest Loss {year}")

# # Display the map
# Map

In [ ]:
# Create a mask for the current year
year = 2009  # Replace with the desired year
loss_year_mask = loss_year.eq(year)
modified_loss_year = loss_year_mask.where(loss_year_mask.eq(year), 1)

# Calculate the loss area for the current year in square meters
loss_area_image = modified_loss_year.multiply(ee.Image.pixelArea())

# Calculate the mean elevation for each country
totalLoss_area = loss_area_image.reduceRegions(
    collection=colombiaMpios,  # The Mpio FeatureCollection
    reducer=ee.Reducer.sum(),  # Use the sum reducer
    scale=30 
)

# Display the first few rows of the DataFrame
#print(totalLoss_area.getInfo())